<a href="https://colab.research.google.com/github/bcury/Machine-Learning-Projects/blob/main/ML_spark_classifica%C3%A7%C3%A3o_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 39.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=3cd8d2e28d038e0157b6dde2fc093b5896858c36fc5bbe3aa30f69b6c929207c
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
import pyspark, findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('NB').getOrCreate()

In [9]:
iris = spark.read.csv('iris.csv', header=True, inferSchema=True, sep=',')
iris.show()

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|
|        5.4|       3.7|        1.5|       0.2|Iris-setosa|
|        4.8|       3.4|        1.6|       0.2|Iris-setosa|
|        4.8|       3.0|        1.4|       0.1|Iris-setosa|
|        4.3|       3.0|        1.1|    

In [12]:
from pyspark.ml.feature import RFormula
Rformula = RFormula(formula='class ~ .', featuresCol='independente', labelCol='dependente')
iris_rformula = Rformula.fit(iris).transform(iris)
iris_rformula.select('independente','dependente').show(truncate=False)

+-----------------+----------+
|independente     |dependente|
+-----------------+----------+
|[5.1,3.5,1.4,0.2]|0.0       |
|[4.9,3.0,1.4,0.2]|0.0       |
|[4.7,3.2,1.3,0.2]|0.0       |
|[4.6,3.1,1.5,0.2]|0.0       |
|[5.0,3.6,1.4,0.2]|0.0       |
|[5.4,3.9,1.7,0.4]|0.0       |
|[4.6,3.4,1.4,0.3]|0.0       |
|[5.0,3.4,1.5,0.2]|0.0       |
|[4.4,2.9,1.4,0.2]|0.0       |
|[4.9,3.1,1.5,0.1]|0.0       |
|[5.4,3.7,1.5,0.2]|0.0       |
|[4.8,3.4,1.6,0.2]|0.0       |
|[4.8,3.0,1.4,0.1]|0.0       |
|[4.3,3.0,1.1,0.1]|0.0       |
|[5.8,4.0,1.2,0.2]|0.0       |
|[5.7,4.4,1.5,0.4]|0.0       |
|[5.4,3.9,1.3,0.4]|0.0       |
|[5.1,3.5,1.4,0.3]|0.0       |
|[5.7,3.8,1.7,0.3]|0.0       |
|[5.1,3.8,1.5,0.3]|0.0       |
+-----------------+----------+
only showing top 20 rows



In [13]:
iris_treino, iris_teste = iris_rformula.randomSplit([0.8,0.2])
iris_treino.count(), iris_teste.count()

(114, 36)

In [14]:
from pyspark.ml.classification import NaiveBayes

In [17]:
NB = NaiveBayes(smoothing=1.0,modelType='multinomial',featuresCol='independente', labelCol='dependente')

In [18]:
modelo = NB.fit(iris_treino)

In [19]:
previsao = modelo.transform(iris_teste)
previsao.select('dependente','prediction').show()

+----------+----------+
|dependente|prediction|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       1.0|       1.0|
|       1.0|       1.0|
|       1.0|       1.0|
|       2.0|       2.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       2.0|       2.0|
|       1.0|       1.0|
+----------+----------+
only showing top 20 rows



In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [25]:
performance = MulticlassClassificationEvaluator(labelCol='dependente',predictionCol='prediction', metricName='accuracy')

In [26]:
accuracy = performance.evaluate(previsao)
print(accuracy)

1.0


modelo perfeito WOW, 100% de eficácia